In [1]:
import pandas as pd
import numpy as np
from helper_functions import *
from constants import *
import re

from sklearn.preprocessing import MinMaxScaler

### Load Data
- remove empty column
- clean column names

In [2]:
df = pd.read_excel("data/Canadian Radio Playlist_V.21.xlsx", "Campus Radio Charts")
df.drop('Unnamed: 24', axis=1, inplace=True)   # Drop empty colummn
df.columns = [re.sub('\s-\s|\s+|-', '_',x.lower().strip()) for x in df.columns]  #Clean column names


df.head(1)

/home/owen/Programs/anaconda3/envs/lighthouse/lib/python3.8/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,week_of,station,station_city,station_province,station_latitude,station_longitude,chart_position,artist_name(s),artist_country,artist_home_city,...,label_name,label_type,language_of_music,visible_ethnic_minority,census_race_classification,artist_gender,m_music,a_artist,p_performance,l_lyrics
0,2006-01-10 00:00:00,CJSR,Edmonton,AB,53.55,-113.5,4,Breakestra,US,"Los Angeles, CA",...,Ubiquity,Indie,English,Yes,Mixed Group,Male Group,No,No,No,No


### Strings
- remove extra white spaces
- lowercase
- convert placeholders to nan values

In [3]:

df = df.applymap(lambda s: re.sub('\s+',' ',s.strip().lower()) if type(s) == str else s)
df.replace(PLACEHOLDERS, np.nan, inplace=True)

### Date Times

- Change '3/2/1010' to '2010-03-02'
- Convert to datetime object

In [4]:
df.loc[df['week_of'] == '3/2/1010', 'week_of'] = '2010-03-02'
df['week_of'] = pd.to_datetime(df['week_of'])

### Change home cities with multiple entries to be only the first entry
- example: "Toronto, Canada/Kingston, Jamaica" would become "Toronto, Canada"

In [5]:
df['artist_home_city'] = df['artist_home_city'].str.split('/').str[0]

### Split 'various' artists into unique groupings
- although the artists are not known, those with the same demographic features and locations are grouped and considered the same

In [6]:
df = make_unique(df,'artist_name(s)',VALUE_EXCEPTIONS,ARTIST_COLUMNS)

### Synchronize Data by key identifiers
- 'identifiers' in this case are Artist Name, Station Name, or Album Name.  It is  assumed than the corresponding information to these identifiers should always be the same.
- Remove any records with missing identifiers
- Ensure that each instance of a station, artist, or ablum always has the same corresponding information

In [7]:
df = df[~df['artist_name(s)'].isna()]
df = df[~df['station'].isna()]
df = df[~df['album_name'].isna()]

In [8]:
df = syncrhonize_data(df, 'artist_home_city', ['artist_home_latitude', 'artist_home_longitude','artist_country'])
df = syncrhonize_data(df, 'station_city', ['station_latitude', 'station_longitude'])
df = syncrhonize_data(df, 'station', STATION_COLUMNS)
df = syncrhonize_data(df, 'artist_name(s)', ARTIST_COLUMNS)
df = syncrhonize_data(df, 'album_name', ALBUM_COLUMNS)

100%|██████████| 2673/2673 [00:23<00:00, 115.57it/s]


### Replace nans in string columns with 'unknown'

In [9]:
df[df.select_dtypes(exclude=np.number).columns] = df.select_dtypes(exclude=np.number).fillna('unknown')

### Give lattitude and longitude to locations without them

In [10]:

df[~(df['artist_home_city'] == 'unknown') & df['artist_home_latitude'].isna()][ARTIST_COLUMNS].drop_duplicates()

,artist_country,artist_home_city,artist_home_latitude,artist_home_longitude,visible_ethnic_minority,census_race_classification,artist_gender,m_music,a_artist,p_performance,l_lyrics
7526,us,california,NaN,NaN,yes,black,male,no,no,no,no
8593,int,germany,NaN,NaN,no,white,male group,no,no,no,no
14606,int,spain,NaN,NaN,no,white,male,no,no,no,no
18732,int,jamaica,NaN,NaN,yes,black,mixed group,no,no,no,no
23036,int,france,NaN,NaN,no,white,male,no,no,no,no
28442,cdn,"fort mcmurray, ab",NaN,NaN,no,white,male,yes,yes,yes,yes


In [11]:
df = assign_lat_long(df, 'fort mcmurray, ab', 56.72, -111.37)
df = assign_lat_long(df, 'california', 36.77, -119.41) #Center of state
df = assign_lat_long(df, 'germany', 52.52, 13.40) # Berlin
df = assign_lat_long(df, 'spain', 40.41, -3.70) #Madrid
df = assign_lat_long(df, 'jamaica', 18.01, -76.80) #Kingston
df = assign_lat_long(df, 'france', 48.85, 2.35) #Paris

### Gender values
- remove group designation from gender and create new column to identify groups from solo artists
- fit all values into 'male, 'mixed', 'female', 'unknown'

In [12]:
df['artist_gender'].value_counts(dropna=False)

male            20180
male group      11765
mixed group      2214
female           1316
unknown           285
female group       97
no                  1
Name: artist_gender, dtype: int64

In [13]:
df['artist_is_group'] = df['artist_gender'].str.contains('group')
df['artist_gender'].replace('male group', 'male', inplace=True)
df['artist_gender'].replace('mixed group', 'mixed', inplace=True)
df['artist_gender'].replace('female group', 'female', inplace=True)
df['artist_gender'].replace('no', 'unknown', inplace=True)
df['artist_gender'].value_counts(dropna=False)

male       31945
mixed       2214
female      1413
unknown      286
Name: artist_gender, dtype: int64

### Visible ethnic minority values
- one instance of "black" is changed to "yes"

In [14]:
df['visible_ethnic_minority'].value_counts(dropna=False)

yes        23965
no         11555
unknown      337
black          1
Name: visible_ethnic_minority, dtype: int64

In [15]:
df['visible_ethnic_minority'].replace('black','yes', inplace=True)
df['visible_ethnic_minority'].value_counts(dropna=False)


yes        23966
no         11555
unknown      337
Name: visible_ethnic_minority, dtype: int64

### Census race classification values
- consolidate similar classifcations into more general categories

In [16]:
df['census_race_classification'].value_counts(dropna=False)

black              15445
white              11546
mixed group         6692
hispanic             499
asian                406
middle eastern       342
unknown              337
native canadian      160
asian indian         132
unidentified          68
other asian           66
indian asian          48
jewish                38
asian other           24
native american       21
inuit                 19
metis                  7
romany                 3
east asian             3
indian                 1
male                   1
Name: census_race_classification, dtype: int64

In [17]:
df['census_race_classification'].replace(['male','unidentified'], 'unknown', inplace=True)
df['census_race_classification'].replace(['asian other','other asian', 'east asian'],'asian', inplace=True)
df['census_race_classification'].replace(['asian indian','indian asian'],'indian', inplace=True)
df['census_race_classification'].replace(['native canadian','native american', 'inuit', 'metis'],'native american', inplace=True)
df['census_race_classification'].value_counts(dropna=False)


black              15445
white              11546
mixed group         6692
hispanic             499
asian                499
unknown              406
middle eastern       342
native american      207
indian               181
jewish                38
romany                 3
Name: census_race_classification, dtype: int64

### Label type
- replace one instance of 'english' to 'unknown'

In [18]:
df['label_type'].value_counts(dropna=False)

indie      27413
major       5027
self        3388
unknown       29
english        1
Name: label_type, dtype: int64

In [19]:
df['label_type'].replace(['english',np.nan], 'unknown', inplace=True)
df['label_type'].value_counts(dropna=False)

indie      27413
major       5027
self        3388
unknown       30
Name: label_type, dtype: int64

### Language of music
- consolidate classifications with multiple langues into 'multiple languages'
- consolidate unknowns

In [20]:
df['language_of_music'].value_counts(dropna=False)

english           34879
french              610
unknown             186
multi                72
english/arabic       38
spanish              20
english/french       13
other                 9
german                8
creole                5
english/patois        4
english/zulu          4
portuguese            4
basque                3
yes                   1
various               1
punjabi               1
Name: language_of_music, dtype: int64

In [21]:
multiple_languages = [  'english/arabic', 
                        'english/french', 
                        'english/patois',
                        'english/zulu', 
                        'various',
                        'multi']
df['language_of_music'].replace(multiple_languages, 'multiple languages', inplace= True)
df['language_of_music'].replace(['yes','other',np.nan], 'unknown', inplace= True)
df['language_of_music'].value_counts(dropna=False)

english               34879
french                  610
unknown                 196
multiple languages      132
spanish                  20
german                    8
creole                    5
portuguese                4
basque                    3
punjabi                   1
Name: language_of_music, dtype: int64

### Distance

- calculate haversine distance where missing and it is possible to

In [22]:
df['km_distance_(home_station)'].isna().value_counts()

False    33987
True      1871
Name: km_distance_(home_station), dtype: int64

In [23]:
row_filter = df['km_distance_(home_station)'].isna() & ~df['artist_home_latitude'].isna()
df.loc[row_filter, 'km_distance_(home_station)'] = haversine(df[row_filter]['station_latitude'],
                                                             df[row_filter]['station_longitude'], 
                                                             df[row_filter]['artist_home_latitude'],
                                                             df[row_filter]['artist_home_longitude'])

In [24]:
df['km_distance_(home_station)'].isna().value_counts()

False    34117
True      1741
Name: km_distance_(home_station), dtype: int64

## Add canadian-content status (met by having at least 2 MAPL ratings)

In [25]:
df['canadian_content'] = (df[['m_music','a_artist','l_lyrics','p_performance']] == 'yes').sum(axis=1) >=2

## Add station_province column

In [26]:
df['station_city'].unique()

array(['edmonton', 'calgary', 'lethbridge', 'victoria', 'burnaby',
       'winnipeg', 'halifax', 'toronto', 'mississauga', 'london',
       'ottawa', 'thunder bay', 'montreal', 'kamloops', 'guelph',
       'saint john', 'windsor', 'fredericton', 'laval', 'nanaimo',
       'smithers'], dtype=object)

In [27]:
province_dictionary = {
                        'edmonton' : 'alberta',
                        'calgary' : 'alberta',
                        'lethbridge': 'alberta',
                        'victoria' : 'british columbia',
                        'burnaby' : 'british columbia',
                        'winnipeg' : 'manitoba',
                        'halifax' : 'nova scotia',
                        'toronto' : 'ontario',
                        'mississauga' : 'ontario',
                        'london' : 'ontario',
                        'ottawa' : 'ontario',
                        'thunder bay' : 'ontario',
                        'montreal' : 'quebec',
                        'kamloops' : 'british columbia',
                        'guelph' : 'ontario',
                        'saint john' : 'new brunswick',
                        'windsor' : 'ontario',
                        'fredericton' : 'new brunswick',
                        'laval' : 'quebec',
                        'nanaimo' : 'british columbia',
                        'smithers' : 'british columbia'
                        }

df['station_province'] = df['station_city'].map(province_dictionary)

In [28]:
df['station_province'].value_counts(normalize=True)

ontario             0.403229
alberta             0.170673
quebec              0.166769
british columbia    0.153355
manitoba            0.066373
nova scotia         0.021752
new brunswick       0.017848
Name: station_province, dtype: float64

In [29]:
population_dictionary = {
                        'edmonton' : 932550,
                        'calgary' : 1239000,
                        'lethbridge': 92730,
                        'victoria' : 85795,
                        'burnaby' : 232755,
                        'winnipeg' : 705245,
                        'halifax' : 403130,
                        'toronto' : 2732000,
                        'mississauga' : 721600,
                        'london' : 383825,
                        'ottawa' : 934240,
                        'thunder bay' : 107910,
                        'montreal' : 1705000,
                        'kamloops' : 90280,
                        'guelph' : 131795,
                        'saint john' : 67575,
                        'windsor' : 217185,
                        'fredericton' : 58220,
                        'laval' : 422995,
                        'nanaimo' : 90505,
                        'smithers' : 5351
                        }

df['city_population'] = df['station_city'].map(population_dictionary)

# Save data

In [30]:
df.to_csv('data/clean_data.csv')

# Station Table

In [31]:
df.columns

Index(['week_of', 'station', 'station_city', 'station_province',
       'station_latitude', 'station_longitude', 'chart_position',
       'artist_name(s)', 'artist_country', 'artist_home_city',
       'artist_home_latitude', 'artist_home_longitude',
       'km_distance_(home_station)', 'album_name', 'label_name', 'label_type',
       'language_of_music', 'visible_ethnic_minority',
       'census_race_classification', 'artist_gender', 'm_music', 'a_artist',
       'p_performance', 'l_lyrics', 'artist_is_group', 'canadian_content',
       'city_population'],
      dtype='object')

In [32]:
station_df = df[['station', 'station_city','city_population','station_province','station_latitude','station_longitude']].drop_duplicates().reset_index(drop=True)

In [33]:
station_df['total_plays'] = station_df['station'].map(df['station'].value_counts())

In [34]:
station_df['total_artists'] = [ df[df['station'] == x ]['artist_name(s)'].nunique() for x in station_df['station'].unique()]

In [35]:
station_df['artists_to_plays_ratio'] = station_df['total_artists'] / station_df['total_plays'] 

In [36]:
station_df['artists_to_population_ratio'] = station_df['total_artists'] / station_df['city_population'] 

In [37]:
station_df['bipoc_artists'] = [[ df[df['station'] == x ]['visible_ethnic_minority'].value_counts(normalize=True)][0]['yes'] for x in station_df['station'].unique()]

In [38]:
station_df['canadian_artists'] =  [[ df[df['station'] == x ]['canadian_content'].value_counts(normalize=True)][0][True] for x in station_df['station'].unique()]

In [39]:
station_df['male_artists'] =  [[ df[df['station'] == x ]['artist_gender'].value_counts(normalize=True)][0]['male'] for x in station_df['station'].unique()]

In [40]:
station_df['english_plays'] =  [[ df[df['station'] == x ]['language_of_music'].value_counts(normalize=True)][0]['english'] for x in station_df['station'].unique()]

In [41]:
station_df.head(5)

,station,station_city,city_population,station_province,station_latitude,station_longitude,total_plays,total_artists,artists_to_plays_ratio,artists_to_population_ratio,bipoc_artists,canadian_artists,male_artists,english_plays
0,cjsr,edmonton,932550,alberta,53.55,-113.50,1920,425,0.221354,0.000456,0.651042,0.432292,0.851562,0.986458
1,cjsw,calgary,1239000,alberta,51.08,-114.08,1970,505,0.256345,0.000408,0.661929,0.303553,0.809645,0.955838
2,ckxu,lethbridge,92730,alberta,49.70,-112.83,2230,539,0.241704,0.005813,0.583408,0.513901,0.885650,0.982960
3,cfuv,victoria,85795,british columbia,48.43,-123.35,2129,488,0.229216,0.005688,0.705965,0.276186,0.882574,0.984969
4,cjsf,burnaby,232755,british columbia,49.25,-123.13,1350,301,0.222963,0.001293,0.729630,0.319259,0.903704,0.988148


In [42]:
station_df.to_csv('data/clean_data_stations.csv')

# Artist Table

In [43]:
df.columns

Index(['week_of', 'station', 'station_city', 'station_province',
       'station_latitude', 'station_longitude', 'chart_position',
       'artist_name(s)', 'artist_country', 'artist_home_city',
       'artist_home_latitude', 'artist_home_longitude',
       'km_distance_(home_station)', 'album_name', 'label_name', 'label_type',
       'language_of_music', 'visible_ethnic_minority',
       'census_race_classification', 'artist_gender', 'm_music', 'a_artist',
       'p_performance', 'l_lyrics', 'artist_is_group', 'canadian_content',
       'city_population'],
      dtype='object')

In [44]:
artist_df = df[['artist_name(s)', 'artist_country','artist_home_city','visible_ethnic_minority','census_race_classification','artist_gender', 'canadian_content', 'artist_is_group']].drop_duplicates().reset_index(drop=True)

In [45]:
artist_df['total_plays'] = artist_df['artist_name(s)'].map(df[['artist_name(s)']].groupby(['artist_name(s)'])['artist_name(s)'].count())

In [46]:
artist_df

,artist_name(s),artist_country,artist_home_city,visible_ethnic_minority,census_race_classification,artist_gender,canadian_content,artist_is_group,total_plays
0,breakestra,us,"los angeles, ca",yes,mixed group,male,False,True,32
1,candy's .22,us,"los angeles, ca",no,white,male,False,True,10
2,dangerdoom,us,"new york, ny",yes,black,male,False,False,18
3,blockhead,us,"new york, ny",no,white,male,False,False,37
4,blackalicious,us,"sacramento, ca",yes,black,male,False,True,2
...,...,...,...,...,...,...,...,...,...
2053,various_69,cdn,"toronto, on",unknown,unknown,unknown,False,False,2
2054,lokz,cdn,"toronto, on",yes,black,male,True,False,1
2055,shad & dallas,cdn,"london, on",yes,black,male,True,False,1
2056,the happy unfortunate,cdn,"winnipeg, mb",yes,black,male,True,False,1


In [47]:
artist_df = pd.merge(artist_df, pd.crosstab(df['artist_name(s)'], df['station_province'], normalize='index'), on='artist_name(s)')

In [48]:
artist_df = pd.concat([artist_df, pd.get_dummies(artist_df[['artist_country', 'visible_ethnic_minority', 'artist_gender']]) ], axis=1 )

In [49]:
artist_df.to_csv('data/clean_data_artists.csv')

# Time Table

In [110]:
df.columns

Index(['week_of', 'station', 'station_city', 'station_province',
       'station_latitude', 'station_longitude', 'chart_position',
       'artist_name(s)', 'artist_country', 'artist_home_city',
       'artist_home_latitude', 'artist_home_longitude',
       'km_distance_(home_station)', 'album_name', 'label_name', 'label_type',
       'language_of_music', 'visible_ethnic_minority',
       'census_race_classification', 'artist_gender', 'm_music', 'a_artist',
       'p_performance', 'l_lyrics', 'artist_is_group', 'canadian_content',
       'city_population'],
      dtype='object')

In [111]:
time_df = df[['week_of']].drop_duplicates().reset_index(drop=True)

In [112]:
time_df['unique_artists_played'] = [df[df['week_of'] == x ]['artist_name(s)'].nunique() for x in time_df['week_of'].unique()]

In [113]:
time_df['total_plays'] = [df[df['week_of'] == x ]['artist_name(s)'].count() for x in time_df['week_of'].unique()]

In [114]:
time_df['bipoc_plays'] = [df[df['week_of'] == x ]['visible_ethnic_minority'].replace(['yes','no','unknown'],[1,0,0]).sum() for x in time_df['week_of'].unique()]

In [115]:
time_df['male_plays'] = [df[df['week_of'] == x ]['artist_gender'].replace(['male','mixed','female','unknown'],[1,0,0,0]).sum() for x in time_df['week_of'].unique()]

In [116]:
time_df['canadian_content_plays'] = [df[df['week_of'] == x ]['canadian_content'].sum() for x in time_df['week_of'].unique()]

In [117]:
for station in df['station'].unique():
    time_df[f'{station}_station_plays'] = [df[(df['week_of'] == x ) & (df['station'] == station)]['artist_name(s)'].count() for x  in time_df['week_of'].unique()]

In [118]:
for province in df['station_province'].unique():
    time_df[f'{station}_province_plays'] = [df[(df['week_of'] == x ) & (df['station'] == province)]['artist_name(s)'].count() for x  in time_df['week_of'].unique()]

In [120]:
time_df

,week_of,unique_artists_played,total_plays,bipoc_plays,male_plays,canadian_content_plays,cjsr_station_plays,cjsw_station_plays,ckxu_station_plays,cfuv_station_plays,...,cjam_station_plays,chsr_station_plays,chry_station_plays,cjlo_station_plays,chyz_station_plays,chly_station_plays,cscr_station_plays,cism_station_plays,cick_station_plays,cick_province_plays
0,2006-01-10,80,130,92,119,32,10,10,10,10,...,0,0,0,0,0,0,0,0,0,0
1,2006-01-17,71,130,86,117,43,0,10,10,10,...,0,0,0,0,0,0,0,0,0,0
2,2006-01-24,72,130,82,116,56,10,10,10,0,...,0,0,0,0,0,0,0,0,0,0
3,2006-01-31,88,150,91,137,55,10,10,10,10,...,0,0,0,0,0,0,0,0,0,0
4,2006-02-07,53,80,49,72,26,10,10,10,10,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
262,2011-02-08,66,110,80,102,63,10,10,10,10,...,10,0,0,10,10,0,0,0,0,0
263,2011-02-15,63,120,73,114,74,10,10,10,10,...,10,0,0,10,10,0,0,0,0,0
264,2011-02-22,54,100,59,94,52,10,10,0,10,...,10,0,0,10,10,0,0,0,0,0
265,2011-03-01,55,100,56,95,57,0,0,10,10,...,0,0,0,10,10,0,0,0,0,0


In [121]:
time_df.to_csv('data/time_table.csv')